In [ ]:
# import pytesseract
# from PIL import Image, ImageEnhance, ImageFilter

# # Load the image
# image_path = 'test.jpg'
# image = Image.open(image_path)

# # Convert the image to grayscale
# image = image.convert('L')

# # Enhance sharpness and apply a filter to remove noise
# image = image.filter(ImageFilter.SHARPEN)
# image = image.filter(ImageFilter.MedianFilter())
# image = image.resize((image.width * 2, image.height * 2), Image.ANTIALIAS)

# # You can also increase contrast if needed
# enhancer = ImageEnhance.Contrast(image)
# image = enhancer.enhance(2)

# # Perform OCR to extract all text
# extracted_text = pytesseract.image_to_string(image)

# # Display the extracted text
# print("Extracted Text from the Image:")
# print(extracted_text)
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

# Load the image
image_path = 'test.jpg'
image = Image.open(image_path)

# Convert the image to grayscale
image = image.convert('L')

# Enhance sharpness and apply a filter to remove noise
image = image.filter(ImageFilter.SHARPEN)
image = image.filter(ImageFilter.MedianFilter())

# Resize the image to double its original size using LANCZOS filter
image = image.resize((image.width * 2, image.height * 2), Image.LANCZOS)

# Increase contrast if needed
enhancer = ImageEnhance.Contrast(image)
image = enhancer.enhance(2)

# Perform OCR to extract all text
extracted_text = pytesseract.image_to_string(image)

# Display the extracted text
print("Extracted Text from the Image:")
print(extracted_text)

